# Spatial Weights


Spatial weights are mathematical structures used to represent spatial relationships. Many spatial analytics, such as spatial autocorrelation statistics and regionalization algorithms rely on spatial weights. Generally speaking, a spatial weight $w_{i,j}$ expresses the notion of a geographical relationship between locations $i$ and $j$. These relationships can be based on a number of criteria including contiguity, geospatial distance and general distances.

PySAL offers functionality for the construction, manipulation, analysis, and conversion of a wide array of spatial weights.

We begin with construction of weights from common spatial data formats.


In [ ]:
import libpysal 
from libpysal.weights import Queen, Rook, KNN
import numpy as np
import geopandas

There are functions to construct weights directly from a file path. 

In [ ]:
shp_path = "data/texas.shp"

## Weight Types

### Contiguity: 
#### Queen Weights

A commonly-used type of weight is a queen contigutiy weight, which reflects adjacency relationships as a binary indicator variable denoting whether or not a polygon shares an edge or a vertex with another polygon. These weights are symmetric, in that when polygon $A$ neighbors polygon $B$, both $w_{AB} = 1$ and $w_{BA} = 1$.

To construct queen weights from a shapefile, we will use geopandas to read the file into a GeoDataFrame, and then use   libpysal to construct the weights:

In [ ]:
df = geopandas.read_file(shp_path)

In [ ]:
qW = Queen.from_dataframe(df)

In [ ]:
qW

All weights objects have a few traits that you can use to work with the weights object, as well as to get information about the weights object. 

To get the neighbors & weights around an observation, use the observation's index on the weights object, like a dictionary:

In [ ]:
qW[4] #neighbors & weights of the 5th observation (0-index remember)

By default, the weights and the pandas dataframe will use the same index. So, we can view the observation and its neighbors in the dataframe by putting the observation's index and its neighbors' indexes together in one list:

In [ ]:
self_and_neighbors = [4]
self_and_neighbors.extend(qW.neighbors[4])
print(self_and_neighbors)

and grabbing those elements from the dataframe:

In [ ]:
df.loc[self_and_neighbors]

A full, dense matrix describing all of the pairwise relationships is constructed using the `.full` method, or when `pysal.full` is called on a weights object:

In [ ]:
Wmatrix, ids = qW.full()
#Wmatrix, ids = ps.full(qW)

In [ ]:
Wmatrix

In [ ]:
n_neighbors = Wmatrix.sum(axis=1) # how many neighbors each region has

In [ ]:
n_neighbors[4]

In [ ]:
qW.cardinalities[4]

Note that this matrix is binary, in that its elements are either zero or one, since an observation is either a neighbor or it is not a neighbor. 

However, many common use cases of spatial weights require that the matrix is row-standardized. This is done simply in PySAL using the `.transform` attribute

In [ ]:
qW.transform = 'r'

Now, if we build a new full matrix, its rows should sum to one:

In [ ]:
Wmatrix, ids = qW.full()

In [ ]:
Wmatrix.sum(axis=1) #numpy axes are 0:column, 1:row, 2:facet, into higher dimensions

Since weight matrices are typically very sparse, there is also a sparse weights matrix constructor:

In [ ]:
qW.sparse

In [ ]:
qW.pct_nonzero #Percentage of nonzero neighbor counts

By default, PySAL assigns each observation an index according to the order in which the observation was read in. This means that, by default, all of the observations in the weights object are indexed by table order. If you have an alternative ID variable, you can pass that into the weights constructor. 

For example, the `texas.shp` dataset has a possible alternative ID Variable, a `FIPS` code.

In [ ]:
df.head()

The observation we were discussing above is in the fifth row: Ochiltree county, Texas. Note that its FIPS code is 48357.

Then, instead of indexing the weights and the dataframe just based on read-order, use the `FIPS` code as an index:

In [ ]:
qW = libpysal.weights.Queen.from_dataframe(df, idVariable='FIPS')

In [ ]:
qW[4] #fails, since no FIPS is 4. 

Note that a `KeyError` in Python usually means that some index, here `4`, was not found in the collection being searched, the IDs in the queen weights object. This makes sense, since we explicitly passed an `idVariable` argument, and nothing has a `FIPS` code of 4.

Instead, if we use the observation's `FIPS` code:

In [ ]:
qW['48357']

We get what we need.

In addition, we have to now query the dataframe using the `FIPS` code to find our neighbors. But, this is relatively easy to do, since pandas will parse the query by looking into python objects, if told to. 

First, let us store the neighbors of our target county:

In [ ]:
self_and_neighbors = ['48357']
self_and_neighbors.extend(qW.neighbors['48357'])

Then, we can use this list in `.query`: 

In [ ]:
df.query('FIPS in @self_and_neighbors')

Note that we have to use `@` before the name in order to show that we're referring to a python object and not a column in the dataframe. 

In [ ]:
#dataframe.query('FIPS in self_and_neighbors') will fail because there is no column called 'self_and_neighbors'

Of course, we could also reindex the dataframe to use the same index as our weights:

In [ ]:
fips_frame = df.set_index(df.FIPS)
fips_frame.head()

Now that both are using the same weights, we can use the `.loc` indexer again:

In [ ]:
fips_frame.loc[self_and_neighbors]

#### Rook Weights

Rook weights are another type of contiguity weight, but consider observations as neighboring only when they share an edge. The rook neighbors of an observation may be different than its queen neighbors, depending on how the observation and its nearby polygons are configured. 

We can construct this in the same way as the queen weights, using the special `rook_from_shapefile` function:

In [ ]:
rW = Rook.from_dataframe(df, idVariable='FIPS')

In [ ]:
rW['48357']

These weights function exactly like the Queen weights, and are only distinguished by what they consider "neighbors."

In [ ]:
self_and_neighbors = ['48357']
self_and_neighbors.extend(rW.neighbors['48357'])
fips_frame.loc[self_and_neighbors]

#### Bishop Weights

In theory, a "Bishop" weighting scheme is one that arises when only polygons that share vertexes are considered to be neighboring. But, since Queen contiguigy requires either an edge or a vertex and Rook contiguity requires only shared edges, the following relationship is true:

$$ \mathcal{Q} = \mathcal{R} \cup \mathcal{B} $$

where $\mathcal{Q}$ is the set of neighbor pairs *via* queen contiguity, $\mathcal{R}$ is the set of neighbor pairs *via* Rook contiguity, and $\mathcal{B}$ *via* Bishop contiguity. Thus:

$$ \mathcal{Q} \setminus \mathcal{R} = \mathcal{B}$$

Bishop weights entail all Queen neighbor pairs that are not also Rook neighbors.

PySAL does not have a dedicated bishop weights constructor, but you can construct very easily using the `w_difference` function. This function is one of a family of tools to work with weights, all defined in `ps.weights`, that conduct these types of set operations between weight objects.

In [ ]:
bW = libpysal.weights.w_difference(qW, rW, constrained=False)

In [ ]:
bW = libpysal.weights.w_difference(qW, rW, constrained=False)

In [ ]:
bW.histogram

Thus, the vast majority of counties have no bishop neighbors. But, a few do. A simple way to see these observations in the dataframe is to find all elements of the dataframe that are not "islands," the term for an observation with no neighbors:

In [ ]:
islands = bW.islands

In [ ]:
# Using `.head()` to limit the number of rows printed
dataframe.query('FIPS not in @islands').head()

## Distance

There are many other kinds of weighting functions in PySAL. Another separate type use a continuous measure of distance to define neighborhoods. 

In [ ]:
radius = libpysal.cg.sphere.RADIUS_EARTH_MILES
radius

In [ ]:
#ps.min_threshold_dist_from_shapefile?

In [ ]:
threshold = libpysal.weights.min_threshold_dist_from_shapefile('data/texas.shp',radius) # now in miles, maximum nearest neighbor distance between the n observations

In [ ]:
threshold

In [ ]:
libpysal.weights.min_threshold_dist_from_shapefile('data/texas.shp')

### knn defined weights

In [ ]:
knn4_bad = KNN.from_dataframe(df, k=4) # ignore curvature of the earth

In [ ]:
knn4_bad.histogram

In [ ]:
knn4 = KNN.from_dataframe(df, k=4, radius=radius)

In [ ]:
knn4.histogram

In [ ]:
knn4[0]

In [ ]:
knn4_bad[0]

In [ ]:
knn4_bad.neighbors == knn4.neighbors

In [ ]:
radius

#### Kernel W

Kernel Weights are continuous distance-based weights that use kernel densities to define the neighbor relationship.
Typically, they estimate a `bandwidth`, which is a parameter governing how far out observations should be considered neighboring. Then, using this bandwidth, they evaluate a continuous kernel function to provide a weight between 0 and 1.

Many different choices of kernel functions are supported, and bandwidths can either be fixed (constant over all units) or adaptive in function of unit density.

For example, if we want to use adaptive bandwidths for the map and weight according to a gaussian kernel:

In [ ]:
cid=155
kernelWa = libpysal.weights.Kernel.from_dataframe(df, k=25, fixed=False, function='gaussian')
df.assign(weights=kernelWa.sparse[cid,:].toarray().flatten()).plot('weights', cmap='plasma')


In [ ]:
kernelWa.cardinalities

In [ ]:
cid=155
kernelWa = libpysal.weights.Kernel.from_dataframe(df, k=25, fixed=False, function='gaussian')
df.assign(weights=kernelWa.sparse[cid,:].toarray().flatten()).plot('weights', cmap='plasma')


In [ ]:
kernelWa = libpysal.weights.Kernel.from_dataframe(df, fixed=True, function='gaussian')

df.assign(weights=kernelWa.sparse[cid,:].toarray().flatten()).plot('weights', cmap='plasma')


In [ ]:
cid=155
kernelWa = libpysal.weights.Kernel.from_dataframe(df, k=50, fixed=False, function='gaussian')
df.assign(weights=kernelWa.sparse[cid,:].toarray().flatten()).plot('weights', cmap='plasma')


In [ ]:
cid=155
kernelWa = libpysal.weights.Kernel.from_dataframe(df, k=10, fixed=False, function='gaussian')
df.assign(weights=kernelWa.sparse[cid,:].toarray().flatten()).plot('weights', cmap='plasma')


In [ ]:
dataframe.loc[kernelWa.neighbors[cid]]

In [ ]:
kernelWa.bandwidth[0:7]

In [ ]:
df.assign(bw=kernelWa.bandwidth.flatten()).plot('bw', cmap='plasma')


In [ ]:
kernelWa.bandwidth[cid]

In [ ]:
kernelWa[cid]

## Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pylab import figure, scatter, show

In [ ]:
wq = ps.queen_from_shapefile('data/texas.shp')

In [ ]:
wq[0]

In [ ]:
fig = figure(figsize=(9,9))
plt.plot(centroids[:,0], centroids[:,1],'.')
plt.ylim([25,37])
show()

In [ ]:
wq.neighbors[0]

In [ ]:
from pylab import figure, scatter, show
fig = figure(figsize=(9,9))

plt.plot(centroids[:,0], centroids[:,1],'.')
#plt.plot(s04[:,0], s04[:,1], '-')
plt.ylim([25,37])
for k,neighs in wq.neighbors.items():
    #print(k,neighs)
    origin = centroids[k]
    for neigh in neighs:
        segment = centroids[[k,neigh]]
        plt.plot(segment[:,0], segment[:,1], '-')
plt.title('Queen Neighbor Graph')
show()

In [ ]:
wr = ps.rook_from_shapefile('data/texas.shp')

In [ ]:
fig = figure(figsize=(9,9))

plt.plot(centroids[:,0], centroids[:,1],'.')
#plt.plot(s04[:,0], s04[:,1], '-')
plt.ylim([25,37])
for k,neighs in wr.neighbors.items():
    #print(k,neighs)
    origin = centroids[k]
    for neigh in neighs:
        segment = centroids[[k,neigh]]
        plt.plot(segment[:,0], segment[:,1], '-')
plt.title('Rook Neighbor Graph')
show()

In [ ]:
fig = figure(figsize=(9,9))
plt.plot(centroids[:,0], centroids[:,1],'.')
#plt.plot(s04[:,0], s04[:,1], '-')
plt.ylim([25,37])
for k,neighs in Wmind.neighbors.items():
    origin = centroids[k]
    for neigh in neighs:
        segment = centroids[[k,neigh]]
        plt.plot(segment[:,0], segment[:,1], '-')
plt.title('Minimum Distance Threshold Neighbor Graph')
show()

In [ ]:
Wmind.pct_nonzero

In [ ]:
wr.pct_nonzero

In [ ]:
wq.pct_nonzero

## Exercise

1. Answer this question before writing any code: What spatial weights structure would be more dense, Texas counties based on rook contiguity or Texas counties based on knn with k=4?
2. Why?
3. Write code to see if you are correct.

## Solution

In [ ]:
wrk = ps.rook_from_shapefile("data/texas.shp")

In [ ]:
wrk.pct_nonzero

In [ ]:
wk4 = ps.knnW_from_shapefile("data/texas.shp", k=4)
wk4.pct_nonzero